In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from LTH_helper import LTH
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import optimizers, initializers
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow_model_optimization.sparsity.keras import prune_low_magnitude, ConstantSparsity, strip_pruning
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
X = train_images.reshape(-1, 784)/255
y = train_labels.reshape(-1, 1)
X_test = test_images.reshape(-1, 784)/255
y_test = test_labels.reshape(-1, 1)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42)

In [11]:
def compile_model(model, lr=0.001):
    optim = optimizers.Adam(lr=lr)
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
    
def get_model(compile_model_flag=True, lr=0.001):
    reduce = 2
    input_dim=784
    output_size = 10
#     initializer = initializers.RandomNormal(mean=0, stddev=0.1)
    initializer = initializers.glorot_normal()
    # Creo el modelo
    model = Sequential()
    model.add(Dense(784*2//reduce, activation='relu', 
                    name='hidden_1', 
                    kernel_initializer=initializer,
                    input_dim=input_dim))
    model.add(Dense(784//reduce, activation='relu', 
                    name='hidden_2', 
                    kernel_initializer=initializer))
    model.add(Dense(output_size, activation='softmax', 
                    name='Salida', 
                    kernel_initializer=initializer))
    if compile_model_flag:
        compile_model(model, lr=lr)
    return model

In [5]:
# lth = LTH(model)

In [9]:
def get_callbacks(filename):
    return [
#         ReduceLROnPlateau(monitor='val_accuracy', mode='max', factor=np.sqrt(0.1), patience=10, verbose=1, min_lr=1e-4),
        ModelCheckpoint(filepath=filename,  verbose=1, save_best_only=True, monitor='val_accuracy', mode='max')
    ]

In [84]:
lth = LTH(get_model)
pruned_model = lth.get_prunned_model('best_untrained/best_untrained_acc_27.9_81.3_not_sparse.hdf5', pm=0.27933673469387754)
compile_model(pruned_model)

In [85]:
lth.test_model_sparsity(pruned_model)

prune_low_magnitude_hidden_1: 0.7206632653061225, 0.27933673469387754
prune_low_magnitude_hidden_2: 0.7206632653061225, 0.27933673469387754
prune_low_magnitude_Salida: 0.7206632653061225, 0.27933673469387754


In [86]:
epochs = 10
batch_size = 256
# Save initial weights
history = pruned_model.fit(X_train, 
           y_train,
           epochs=epochs, batch_size=batch_size, 
           validation_data = (X_val, y_val),
           verbose=2, 
           callbacks=get_callbacks('mlp.mnist.first_train.hdf5')
          )

Epoch 1/10

Epoch 00001: val_accuracy improved from -inf to 0.89278, saving model to mlp.mnist.first_train.hdf5
200/200 - 3s - loss: 0.2281 - accuracy: 0.9148 - val_loss: 0.2910 - val_accuracy: 0.8928
Epoch 2/10

Epoch 00002: val_accuracy improved from 0.89278 to 0.89300, saving model to mlp.mnist.first_train.hdf5
200/200 - 3s - loss: 0.2011 - accuracy: 0.9246 - val_loss: 0.2933 - val_accuracy: 0.8930
Epoch 3/10

Epoch 00003: val_accuracy improved from 0.89300 to 0.89900, saving model to mlp.mnist.first_train.hdf5
200/200 - 3s - loss: 0.1901 - accuracy: 0.9277 - val_loss: 0.2796 - val_accuracy: 0.8990
Epoch 4/10

Epoch 00004: val_accuracy improved from 0.89900 to 0.89933, saving model to mlp.mnist.first_train.hdf5
200/200 - 3s - loss: 0.1805 - accuracy: 0.9330 - val_loss: 0.2895 - val_accuracy: 0.8993
Epoch 5/10

Epoch 00005: val_accuracy did not improve from 0.89933
200/200 - 3s - loss: 0.1680 - accuracy: 0.9374 - val_loss: 0.2964 - val_accuracy: 0.8959
Epoch 6/10

Epoch 00006: val_ac

In [87]:
lth.test_model_sparsity(pruned_model)

prune_low_magnitude_hidden_1: 0.7206632653061225, 0.27933673469387754
prune_low_magnitude_hidden_2: 0.7206632653061225, 0.27933673469387754
prune_low_magnitude_Salida: 0.7206632653061225, 0.27933673469387754
